In [10]:
import findspark, datetime, pytz
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F

findspark.add_packages('mysql:mysql-connector-java:8.0.11')

In [2]:
# Create SparkSession
spark = (SparkSession.builder
           .appName('SparkByExamples.com')
           .config("packages", "org.apache.spark:mysql-connector-java-8.0.13.jar")
           .getOrCreate()
        )

24/06/08 23:47:51 WARN Utils: Your hostname, cj resolves to a loopback address: 127.0.1.1; using 192.168.15.34 instead (on interface enp2s0)
24/06/08 23:47:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/cj/.ivy2/cache
The jars for the packages stored in: /home/cj/.ivy2/jars
mysql#mysql-connector-java added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-84830ee2-7a4c-491e-8dc2-72fdf769294a;1.0
	confs: [default]
	found mysql#mysql-connector-java;8.0.11 in central
	found com.google.protobuf#protobuf-java;2.6.0 in central
:: resolution report :: resolve 124ms :: artifacts dl 6ms
	:: modules in use:
	com.google.protobuf#protobuf-java;2.6.0 from central in [default]
	mysql#mysql-connector-java;8.0.11 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	--------------------------------------------------

In [28]:
# Read from MySQL Table
VendasDF = (spark.read
        .format("jdbc")
        .option("driver","com.mysql.cj.jdbc.Driver")
        .option("url", "jdbc:mysql://localhost:3306/vendas?allowPublicKeyRetrieval=true&useSSL=false")
        .option("dbtable", 'vendas')
        .option("user", "big_data_importer")
        .option("password", "big_data_importer")
        .load()
     )
ItensVendaDF = (spark.read
        .format("jdbc")
        .option("driver","com.mysql.cj.jdbc.Driver")
        .option("url", "jdbc:mysql://localhost:3306/vendas?allowPublicKeyRetrieval=true&useSSL=false")
        .option("dbtable", 'itens_venda')
        .option("user", "big_data_importer")
        .option("password", "big_data_importer")
        .load()
     )
VendedoresDF = (spark.read
        .format("jdbc")
        .option("driver","com.mysql.cj.jdbc.Driver")
        .option("url", "jdbc:mysql://localhost:3306/vendas?allowPublicKeyRetrieval=true&useSSL=false")
        .option("dbtable", 'vendedores')
        .option("user", "big_data_importer")
        .option("password", "big_data_importer")
        .load()
     )
ClientesDF = (spark.read
        .format("jdbc")
        .option("driver","com.mysql.cj.jdbc.Driver")
        .option("url", "jdbc:mysql://localhost:3306/vendas?allowPublicKeyRetrieval=true&useSSL=false")
        .option("dbtable", 'clientes')
        .option("user", "big_data_importer")
        .option("password", "big_data_importer")
        .load()
     )

In [35]:
idVendas = [3,4,6,7,8,9,11,12,13,14,918]
VendasDF.filter(F.col('id_venda').isin(idVendas)).show()

+--------+-----------+----------+----------+--------+
|id_venda|id_vendedor|id_cliente|      data|   total|
+--------+-----------+----------+----------+--------+
|       3|          1|       300|2015-12-31| 8053.60|
|       4|          1|       300|2015-12-31| 8053.60|
|       6|          8|       500|2024-04-04|20685.00|
|       7|         26|       669|2024-04-04|25530.00|
|       8|         30|       480|2024-04-04|15316.00|
|       9|         12|       482|2024-04-04|29785.00|
|      11|          6|       284|2024-04-04| 2966.00|
|      12|         19|       678|2024-04-04| 1240.00|
|      13|          3|       335|2024-04-04|  675.00|
|      14|         33|       651|2024-04-04|  775.00|
|     918|         27|       253|2024-04-04|  620.00|
+--------+-----------+----------+----------+--------+



In [36]:
idVendas = [3,4,6,7,8,9,11,12,13,14,918]
ItensVendaDF.filter(F.col('id_venda').isin(918)).show()

+----------+--------+----------+--------------+-----------+--------+
|id_produto|id_venda|quantidade|valor_unitario|valor_total|desconto|
+----------+--------+----------+--------------+-----------+--------+
+----------+--------+----------+--------------+-----------+--------+



In [25]:
idVendedores = [1,1,8,26,30,12,6,19,3,33]
VendedoresDF.filter(F.col('id_vendedor').isin(idVendedores)).show()

+-----------+--------------+---------------+--------------------+-------------+--------------------+
|id_vendedor|           cpf|       telefone|               email|origem_racial|                nome|
+-----------+--------------+---------------+--------------------+-------------+--------------------+
|          1|523.120.480-35|(88) 90334-0392|armando_lago@loja...|       branca|        Armando Lago|
|          3|017.474.393-98|(31) 93436-6016|daniel_piraja@loj...|        preta|       Daniel Pirajá|
|          6|416.552.263-55|(22) 93277-8907|ibere_lacerda@loj...|     indígena|       Iberê Lacerda|
|          8|981.475.880-97|(18) 90653-1009|napoleao_mendez@l...|     indígena|     Napoleão Méndez|
|         12|404.314.222-65|(94) 94677-8772|capitolino_bahia@...|      amarela|    Capitolino Bahía|
|         19|075.959.284-51|(19) 96086-8436|simao_rivero@loja...|        preta|        Simão Rivero|
|         26|902.331.481-67|(37) 99504-7180|kevin_santiago@lo...|        preta|Kevin Delato

In [27]:
idClientes = [251,300,300,500,669,480,482,284,678,335,651]
ClientesDF.filter(F.col('id_cliente').isin(idClientes)).show()

+----------+--------------+---------------+--------------------+--------------------+------+-------------+----+------+
|id_cliente|           cpf|       telefone|               email|             cliente|estado|origem_racial|sexo|status|
+----------+--------------+---------------+--------------------+--------------------+------+-------------+----+------+
|       251|802.404.251-70|(82) 96797-3891|adelina.buenavent...|Adelina Buenaventura|    RJ|        preta|   M|Silver|
|       284|096.161.403-02|(71) 96033-3200|antonia.canhao@em...|      Antónia Canhão|    SC|        preta|   F|Silver|
|       300|356.589.970-03|(19) 98218-2227|belmiro.marroquim...|   Belmiro Marroquim|    SC|      amarela|   M|Silver|
|       335|466.049.055-60|(68) 94346-4877| cid.pardo@email.com|           Cid Pardo|    AL|        preta|   M|Silver|
|       480|690.175.659-34|(51) 92030-1486|humberto.vergueir...|  Humberto Vergueiro|    MG|        preta|   M|Silver|
|       482|622.783.526-95|(86) 93297-8927|ifige

In [33]:
ClientesDF.select(F.count('*')).show()

+--------+
|count(1)|
+--------+
|     530|
+--------+

